In [ ]:
df = spark.read.option("multiline", "true").json("dbfs:/FileStore/FileStore/movies-1.json")

In [ ]:
df.printSchema()

root
 |-- cast: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- extract: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- href: string (nullable = true)
 |-- thumbnail: string (nullable = true)
 |-- thumbnail_height: long (nullable = true)
 |-- thumbnail_width: long (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [ ]:
df.show(10)

+-------------+--------------------+--------------------+--------------------+--------------------+----------------+---------------+--------------------+----+
|         cast|             extract|              genres|                href|           thumbnail|thumbnail_height|thumbnail_width|               title|year|
+-------------+--------------------+--------------------+--------------------+--------------------+----------------+---------------+--------------------+----+
|           []|                NULL|                  []|                NULL|                NULL|            NULL|           NULL|After Dark in Cen...|1900|
|           []|                NULL|                  []|                NULL|                NULL|            NULL|           NULL|Boarding School G...|1900|
|           []|                NULL|                  []|                NULL|                NULL|            NULL|           NULL|Buffalo Bill's Wi...|1900|
|           []|                NULL|          

In [ ]:
df.na.drop()

DataFrame[cast: array<string>, extract: string, genres: array<string>, href: string, thumbnail: string, thumbnail_height: bigint, thumbnail_width: bigint, title: string, year: bigint]

In [ ]:
df = df.dropDuplicates(["title", "year"])

In [ ]:
from pyspark.sql.functions import when;
df = df.withColumn("centure", when(df["year"] >= 2000, "21").otherwise("20"))
df.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+---------------+--------------------+----+-------+
|                cast|             extract|              genres|                href|           thumbnail|thumbnail_height|thumbnail_width|               title|year|centure|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+---------------+--------------------+----+-------+
|                  []|                NULL|                  []|                NULL|                NULL|            NULL|           NULL| The Automobile Race|1904|     20|
|                  []|Mrs. Jones Entert...|[Comedy, Short, S...|Mrs._Jones_Entert...|https://upload.wi...|             189|            320|Mrs. Jones Entert...|1909|     20|
|                  []|                NULL|                  []|                NULL|                NULL|            NULL|       

In [ ]:
df_filtered = df.filter((df.year >= 2015))
df_filtered.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+---------------+--------------------+----+-------+
|                cast|             extract|              genres|                href|           thumbnail|thumbnail_height|thumbnail_width|               title|year|centure|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+---------------+--------------------+----+-------+
|[Jesse Eisenberg,...|American Ultra is...|    [Action, Comedy]|      American_Ultra|https://upload.wi...|             380|            256|      American Ultra|2015|     21|
|[Ryan Reynolds, J...|Pokémon Detective...|[Comedy, Fantasy,...|Detective_Pikachu...|https://upload.wi...|             385|            260|Pokémon Detective...|2019|     21|
|[Amy Ryan, Nick R...|Strange but True ...|          [Thriller]|Strange_but_True_...|https://upload.wi...|             383|       

In [ ]:
config = {
    "spark.cosmos.accountEndpoint": "https://it-step-cosmos-db2.documents.azure.com:443/",
    "spark.cosmos.accountKey": "",
    "spark.cosmos.database": "MediaDB",
    "spark.cosmos.container": "Movies"
}

spark.conf.set("spark.sql.catalog.cosmosCatalog", "com.azure.cosmos.spark.CosmosCatalog")
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.accountEndpoint", config["spark.cosmos.accountEndpoint"])
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.accountKey", config["spark.cosmos.accountKey"])

spark.sql("CREATE DATABASE IF NOT EXISTS cosmosCatalog.MediaDB;")

DataFrame[]

In [ ]:
spark.sql("CREATE TABLE IF NOT EXISTS cosmosCatalog.MediaDB.Movies USING cosmos.oltp TBLPROPERTIES(partitionKeyPath = '/category', autoScaleMaxThroughput = '1000')")

DataFrame[]

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

df = df.withColumn("id", monotonically_increasing_id().cast("string"))
df.show(5)


+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+---------------+--------------------+----+-------+---+
|                cast|             extract|              genres|                href|           thumbnail|thumbnail_height|thumbnail_width|               title|year|centure| id|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+---------------+--------------------+----+-------+---+
|                  []|                NULL|                  []|                NULL|                NULL|            NULL|           NULL| The Automobile Race|1904|     20|  0|
|                  []|Mrs. Jones Entert...|[Comedy, Short, S...|Mrs._Jones_Entert...|https://upload.wi...|             189|            320|Mrs. Jones Entert...|1909|     20|  1|
|                  []|                NULL|                  []|                NULL|                NULL|    

In [ ]:
writeConfig = {
    "spark.cosmos.accountEndpoint": "https://it-step-cosmos-db2.documents.azure.com:443/",
    "spark.cosmos.accountKey": "",
    "spark.cosmos.database": "MediaDB",
    "spark.cosmos.container": "Movies",
    "spark.cosmos.write.strategy": "ItemOverwrite"
}

df.write.format("cosmos.oltp").options(**writeConfig).mode("Append").save()

In [ ]:
df.count()

36243

In [ ]:
readConfig = {
    "spark.cosmos.accountEndpoint": "https://it-step-cosmos-db2.documents.azure.com:443/",
    "spark.cosmos.accountKey": "",
    "spark.cosmos.database": "MediaDB",
    "spark.cosmos.container": "Movies"
}
df1 = spark.read.format("cosmos.oltp").options(**readConfig).load()

In [ ]:
df1.count()

36243

In [ ]:
df1.show(10)

+-------+----+--------------------+-----------+--------------------+-----------------+---------------+--------------------+--------------------+----------------+--------------------+
|centure|year|           thumbnail|         id|                cast|           genres|thumbnail_width|               title|                href|thumbnail_height|             extract|
+-------+----+--------------------+-----------+--------------------+-----------------+---------------+--------------------+--------------------+----------------+--------------------+
|     20|1939|https://upload.wi...|       1716|[Joe E. Brown, Ma...|         [Comedy]|            259|  $1,000 a Touchdown|  $1,000_a_Touchdown|             385|$1,000 a Touchdow...|
|     20|1935|                NULL|       8120|[Edward Everett H...|         [Comedy]|           NULL|           $10 Raise|           $10_Raise|            NULL|$10 Raise is a 19...|
|     20|1921|                NULL| 8589938995|                  []|       [Animated]

In [ ]:
df1.createOrReplaceTempView("cosmos_view"); spark.sql("SELECT * FROM cosmos_view WHERE year = '2015'").show()

+-------+----+--------------------+-----------+--------------------+--------------------+---------------+--------------------+--------------------+----------------+--------------------+
|centure|year|           thumbnail|         id|                cast|              genres|thumbnail_width|               title|                href|thumbnail_height|             extract|
+-------+----+--------------------+-----------+--------------------+--------------------+---------------+--------------------+--------------------+----------------+--------------------+
|     21|2015|https://upload.wi...|       6072|[Simon Abkarian, ...|          [Thriller]|            256|                1915|         1915_(film)|             380|1915 is a 2015 Am...|
|     21|2015|https://upload.wi...|25769809621|[Amber Tamblyn, W...|             [Drama]|            259|3 Nights in the D...|3_Nights_in_the_D...|             384|3 Nights in the D...|
|     21|2015|https://upload.wi...|       6595|[Andrew Garfield,...|  